In [2]:
import pandas as pd
from  bs4 import BeautifulSoup
import requests
from warnings import filterwarnings
filterwarnings('ignore')
import datetime as dt
from time import sleep

### Краснодар

In [7]:
def parse_minfinkubani_ru():
    domain = 'https://minfinkubani.ru'
    url = 'https://minfinkubani.ru/deposit_funds/selection_parameters.php'
    
    s = requests.Session()
    r = s.get(url, verify=False)
    
    soup = BeautifulSoup(r.text)
    parags = soup.find_all('div', {'class': 'item_mews_title'})
    parsed = []

    for p in parags:
        ext_url = domain + p.a.attrs['href']
        ext_content = BeautifulSoup(requests.get(ext_url, verify=False).text)
        d = ext_url.split('=')[2]
        if dt.datetime.strptime(d, '%d.%m.%Y') >= dt.datetime.today():
            parsed.append([d, ext_url])
            
    s.close()
    parsed = pd.DataFrame(parsed, columns=['date', 'url'])
    
    return parsed

In [8]:
parsed_minfinkubani = parse_minfinkubani_ru()
parsed_minfinkubani

,date,url
0,08.07.2021,https://minfinkubani.ru/deposit_funds/detail.p...
1,29.06.2021,https://minfinkubani.ru/deposit_funds/detail.p...
2,03.06.2021,https://minfinkubani.ru/deposit_funds/detail.p...
3,19.05.2021,https://minfinkubani.ru/deposit_funds/detail.p...
4,29.04.2021,https://minfinkubani.ru/deposit_funds/detail.p...
5,19.04.2021,https://minfinkubani.ru/deposit_funds/detail.p...
6,05.04.2021,https://minfinkubani.ru/deposit_funds/detail.p...
7,22.03.2021,https://minfinkubani.ru/deposit_funds/detail.p...
8,04.03.2021,https://minfinkubani.ru/deposit_funds/detail.p...
9,17.02.2021,https://minfinkubani.ru/deposit_funds/detail.p...


### Банк России

In [9]:
def parse_cbr_ru():
    url = 'https://cbr.ru/DKP/DepoParams/'
    df = pd.read_html(url)[0].iloc[:, 0]
    tod = dt.date.today().strftime("%d.%m.%Y")
    df = df[pd.to_datetime(df, format='%d.%m.%Y') >= tod]
    df = pd.DataFrame({'date':df.to_list(), 'url':[url] * len(df)})
    return df

In [10]:
parsed_cbr = parse_cbr_ru()
parsed_cbr

,date,url
0,20.07.2021,https://cbr.ru/DKP/DepoParams/
1,13.07.2021,https://cbr.ru/DKP/DepoParams/
2,06.07.2021,https://cbr.ru/DKP/DepoParams/
3,06.07.2021,https://cbr.ru/DKP/DepoParams/
4,05.07.2021,https://cbr.ru/DKP/DepoParams/
5,02.07.2021,https://cbr.ru/DKP/DepoParams/
6,29.06.2021,https://cbr.ru/DKP/DepoParams/
7,22.06.2021,https://cbr.ru/DKP/DepoParams/
8,15.06.2021,https://cbr.ru/DKP/DepoParams/


### Moex

In [11]:
def month2num(month):
    month_dict = {
        'января': 1,
        'февраля': 2,
        'марта': 3,
        'апреля': 4,
        'мая':5,
        'июня': 6,
        'июля': 7,
        'августа': 8,
        'сентября': 9,
        'октября': 10,
        'ноября': 11,
        'декабря': 12
    }
    return month_dict[month]

In [12]:
def moex_parser(kwords):
    domain = 'https://www.moex.com/'
    url = 'https://www.moex.com/ru/news/?ncat=114'
    
    r = requests.get(url).text
    soup = BeautifulSoup(r)
    parags = soup.find_all('a', {'class': 'news-list__link'})
    
    parsed = []

    for p in parags:
        full_url = domain + p.attrs['href']
        text = p.text.lower()

        for w in kwords:
            if w not in text:
                break
        else:
            action_date = p.text.lower().split('состоится')[0]
            action_date =\
                f'{action_date.split()[0]}.{month2num(action_date.split()[1])}.{action_date.split()[2]}'
            action_date = dt.datetime.strptime(action_date, '%d.%m.%Y')
            if action_date >= dt.datetime.today():
                parsed.append([dt.datetime.strftime(action_date, '%d.%m.%Y'),
                               full_url
                              ])

    parsed = pd.DataFrame({'date':[x[0] for x in parsed],
                           'url':[x[1] for x in parsed]})
    
    return parsed

In [13]:
words_fedkazna = ['федеральн', 'казначей']
parsed_fedkazna = moex_parser(words_fedkazna)
parsed_fedkazna

,date,url
0,21.07.2021,https://www.moex.com//n35185/?nt=114
1,20.07.2021,https://www.moex.com//n35161/?nt=114
2,14.07.2021,https://www.moex.com//n35082/?nt=114
3,13.07.2021,https://www.moex.com//n35060/?nt=114
4,07.07.2021,https://www.moex.com//n34978/?nt=114
5,06.07.2021,https://www.moex.com//n34964/?nt=114
6,30.06.2021,https://www.moex.com//n34876/?nt=114
7,29.06.2021,https://www.moex.com//n34851/?nt=114
8,23.06.2021,https://www.moex.com//n34736/?nt=114
9,22.06.2021,https://www.moex.com//n34720/?nt=114


In [14]:
words_pfr = ['пенсион', 'фонд']
parsed_pfr = moex_parser(words_pfr)
parsed_pfr

,date,url
0,07.07.2021,https://www.moex.com//n34979/?nt=114
1,25.06.2021,https://www.moex.com//n34782/?nt=114


###  Комитет финансов СПБ

In [124]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from time import sleep

In [128]:
def com_spb_parser(kwords):
    options = Options()
    options.headless = True
    driver = webdriver.Firefox(options=options)
    url = 'https://комфинспб.рф/committees/news/'
    parsed = []
    
    driver.get(url)
    sleep(10)
    html = driver.page_source
    soup = BeautifulSoup(html)
    h3_list = soup.findAll('h3')

    for e in h3_list:
        text = e.text
        url_id = e['data-id']
        news_url = url + url_id

        for k in kwords:
            if k not in text:
                break
        else:
            text, date = text.split(' - ')
            action_date = dt.datetime.strptime(date, '%d.%m.%Y')
            if action_date >= dt.datetime.today():
                    parsed.append([dt.datetime.strftime(action_date, '%d.%m.%Y'),
                                   news_url
                                  ])

    parsed = pd.DataFrame({'date':[x[0] for x in parsed],
                               'url':[x[1] for x in parsed]})
    
    driver.close()

    return parsed

In [129]:
words_comspb = ['провед', 'депозит']
parsed_comspb = com_spb_parser(words_fedkazna)
parsed_comspb

,date,url


### Aggregation

In [130]:
df = parsed_cbr.append(parsed_minfinkubani)
df = df.append(parsed_fedkazna)
df = df.append(parsed_pfr)
df = df.append(parsed_comspb)

In [131]:
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df.sort_values('date')

,date,url
12,2020-12-15,https://minfinkubani.ru/deposit_funds/detail.p...
11,2021-01-18,https://minfinkubani.ru/deposit_funds/detail.p...
10,2021-01-27,https://minfinkubani.ru/deposit_funds/detail.p...
9,2021-02-17,https://minfinkubani.ru/deposit_funds/detail.p...
8,2021-03-04,https://minfinkubani.ru/deposit_funds/detail.p...
7,2021-03-22,https://minfinkubani.ru/deposit_funds/detail.p...
6,2021-04-05,https://minfinkubani.ru/deposit_funds/detail.p...
5,2021-04-19,https://minfinkubani.ru/deposit_funds/detail.p...
4,2021-04-29,https://minfinkubani.ru/deposit_funds/detail.p...
3,2021-05-19,https://minfinkubani.ru/deposit_funds/detail.p...


### Writing

In [18]:
df.to_excel('parsed_deposit_auctions.xlsx')